In [1]:
import os
import yaml

os.chdir('Z:\OMOP\omop_etl')
print(os.getcwd())

import pandas as pd
from omop_etl.load import Loader
# from .omop_etl.stage import STAGE

from pydma.utils import find

Z:\OMOP\omop_etl


In [32]:
omop = Loader('config.yml')

# CONFIG = 'omop_etl/etl_config.yml'
# with open(CONFIG) as f:
#     yml = yaml.safe_load(f)

ALIASES = omop.aliases
STAGE = omop.stage

In [33]:
STAGE

{'condition_occurrence': 'CONDITION',
 'death': 'DEATH',
 'drug_exposure': {'admin': 'DRUG_ADMIN', 'order': 'DRUG_ORDER'},
 'measurement': {'bp': 'MEASUREMENT_BP_BP',
  'bp_artline': 'MEASUREMENT_BP_ArterialLine',
  'bp_noninvasive': 'MEASUREMENT_BP_BP_NonInvasive',
  'bp_cvp': 'MEASUREMENT_BP_CVP',
  'bp_cvp_mean': 'MEASUREMENT_BP_CVP_mean',
  'bp_map_artline': 'MEASUREMENT_BP_MAP_A_line',
  'bp_map_cuff': 'MEASUREMENT_BP_MAP_Cuff',
  'bp_map_noninvasive': 'MEASUREMENT_BP_MAP_NonINvasive',
  'bp_pap_mean': 'MEASUREMENT_BP_PAP_mean',
  'heart_rate': 'MEASUREMENT_HeartRate',
  'height': 'MEASUREMENT_Height',
  'lab': 'MEASUREMENT_LAB',
  'lda': 'MEASUREMENT_LDA',
  'pain': 'MEASUREMENT_PainScale',
  'pain_jax': 'MEASUREMENT_PainScale_JAX',
  'pain_peds': 'MEASUREMENT_PainScale_Peds',
  'qtcb': 'MEASUREMENT_QTCB',
  'res_dev': 'MEASUREMENT_Res_Device',
  'res_etco2': 'MEASUREMENT_Res_ETCO2',
  'res_etco2_on': 'MEASUREMENT_Res_ETCO2_NO',
  'res_fio2': 'MEASUREMENT_Res_FIO2',
  'res_gcs': 

In [15]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Count'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [16]:
stage_t_dict = stage_t[['Table','Count']].to_dict()
# stage_t_dict

In [17]:
stage_t_dict['Count'][7]

5172282

In [18]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [34]:
loaded = omop.config.load

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Count'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Count'][idx]
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Count'])
# table_counts.head(50)

In [36]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['RC_Load'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['RC_DeId'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))
count_diff['RC_Preload'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='preload') if t not in ('death','person','visit_occurrence') else 0)
count_diff

,Table,Count,RC_Load,RC_DeId,RC_Preload
0,condition_occurrence,74725451,61154887,61154102,74743630
1,death,6452,6452,6452,0
2,drug_exposure,79519205,79394081,79393862,79398201
3,measurement,315736168,382511603,382511603,371840188
4,observation,80449088,99003172,99001854,80534624
5,person,261222,261227,261220,0
6,procedure_occurrence,29445698,13389500,13389310,29328512
7,visit_occurrence,26752594,26663500,26663088,0


In [ ]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()